In [57]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as sm


In [58]:
auto = pd.read_csv('Auto.csv', na_values='?')
auto.info()
auto.head()
#auto.horsepower should be float/int, it is object -> na values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           397 non-null    float64
 1   cylinders     397 non-null    int64  
 2   displacement  397 non-null    float64
 3   horsepower    392 non-null    float64
 4   weight        397 non-null    int64  
 5   acceleration  397 non-null    float64
 6   year          397 non-null    int64  
 7   origin        397 non-null    int64  
 8   name          397 non-null    object 
dtypes: float64(4), int64(4), object(1)
memory usage: 28.0+ KB


,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [59]:
auto.horsepower.unique()

array([130., 165., 150., 140., 198., 220., 215., 225., 190., 170., 160.,
        95.,  97.,  85.,  88.,  46.,  87.,  90., 113., 200., 210., 193.,
        nan, 100., 105., 175., 153., 180., 110.,  72.,  86.,  70.,  76.,
        65.,  69.,  60.,  80.,  54., 208., 155., 112.,  92., 145., 137.,
       158., 167.,  94., 107., 230.,  49.,  75.,  91., 122.,  67.,  83.,
        78.,  52.,  61.,  93., 148., 129.,  96.,  71.,  98., 115.,  53.,
        81.,  79., 120., 152., 102., 108.,  68.,  58., 149.,  89.,  63.,
        48.,  66., 139., 103., 125., 133., 138., 135., 142.,  77.,  62.,
       132.,  84.,  64.,  74., 116.,  82.])

In [60]:
auto.horsepower.fillna(auto.horsepower.mean(skipna=True), inplace=True)
auto['origin_str'] = auto.origin.astype(str)
auto['cylinders_str'] = auto.cylinders.astype(str)

In [61]:
#Scaling all quantitative variables only by ensuring float64/int64 type

from sklearn.preprocessing import scale
for i in auto.drop(columns=['mpg', 'horsepower', 'origin', 'cylinders']).columns:
    if pd.api.types.is_float_dtype(auto[i]) or pd.api.types.is_int64_dtype(auto[i]):
        auto[i+'_s'] = scale(auto[i], with_mean=True, with_std=True)
    else:
        auto[i] = auto[i]

In [62]:
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,origin_str,cylinders_str,displacement_s,weight_s,acceleration_s,year_s
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,1,8,1.088435,0.630274,-1.294604,-1.626699
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,1,8,1.500913,0.853457,-1.476652,-1.626699
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,1,8,1.193953,0.549975,-1.658700,-1.626699
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,1,8,1.059658,0.546432,-1.294604,-1.626699
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,1,8,1.040473,0.565326,-1.840747,-1.626699


In [63]:
from statsmodels.tools.tools import add_constant
autoQuant = add_constant(pd.DataFrame({'Displacement': auto.displacement_s, 'Weight': auto.weight_s, 'Acceleration': auto.acceleration_s, 'Year': auto.year_s}))

In [64]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
vif['Features'] = autoQuant.columns
vif['VIF'] = [variance_inflation_factor(autoQuant, i) for i in range(len(autoQuant.columns))]

In [65]:
vif

,Features,VIF
0,const,1.000000
1,Displacement,10.237192
2,Weight,8.517832
3,Acceleration,1.562660
4,Year,1.179748


In [66]:
autoQuant.drop(columns='Displacement', inplace=True)
vif = pd.DataFrame()
vif['Features'] = autoQuant.columns
vif['VIF'] = [variance_inflation_factor(autoQuant, i) for i in range(len(autoQuant.columns))]
vif

,Features,VIF
0,const,1.000000
1,Weight,1.273727
2,Acceleration,1.253278
3,Year,1.140882


In [67]:
sm.ols('mpg~weight_s+year_s', data=auto).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.809
Model:                            OLS   Adj. R-squared:                  0.808
Method:                 Least Squares   F-statistic:                     833.1
Date:                Sat, 22 Apr 2023   Prob (F-statistic):          3.00e-142
Time:                        19:55:31   Log-Likelihood:                -1051.3
No. Observations:                 397   AIC:                             2109.
Df Residuals:                     394   BIC:                             2120.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     23.5159      0.172    136.560      0.000      23.177      23.854
weight_s      -5.6358      0.181    -31.138      0.000      -5.992      -5.280
year_s         2.8095      0.181     15.522      0.000       2.454       3.165
==============================================================================
Omnibus:                       41.321   Durbin-Watson:                   1.222
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               67.926
Skew:                           0.659   Prob(JB):                     1.78e-15
Kurtosis:                       4.538   Cond. No.                         1.37
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [68]:
np.random.seed(4)
trainingIndex = np.random.choice(auto.shape[0], size=318, replace=False)

In [69]:
trainingIndex

array([283, 361, 336,  64,   6, 373, 102, 153, 382,  70,  55, 353, 396,
       293,  11, 167, 120, 139, 216, 200, 381,  14, 302, 124, 300, 271,
        72, 347,  68,  83,  24, 225, 221, 327, 146,  47, 328, 106,  13,
       103, 133, 185, 134,  25, 188, 317, 319, 241, 266,  84, 193, 220,
        92, 267, 242, 314,  80, 386,  61, 159, 129, 352, 251,  34, 261,
        66, 384, 239,  82, 219, 391, 184, 202, 229, 105, 172,  96,  42,
       365, 123, 284, 330, 338,  18, 157, 166, 394,  43,   1, 138, 191,
       389,  63, 100, 315,  60, 160,  12, 128, 318, 275, 362, 309, 180,
       210, 301,  89, 291,  45, 247, 340, 142, 299, 270,  93,  33, 186,
       363, 212, 189,  88, 257, 112, 349, 393, 170, 335,  46, 226, 154,
       235, 130, 324, 285,  99, 233, 332, 211, 175, 280, 272, 137,  16,
       169, 152,  91, 265, 195, 101, 329, 290,  39, 303,  17, 320,  71,
        20, 203, 205, 268, 312, 254, 258, 274, 192, 232, 367, 263,  69,
       256, 213, 173, 182, 206, 246, 113, 368, 325, 289, 223, 31

In [70]:
select= np.in1d(range(auto.shape[0]), trainingIndex)

In [71]:
training = auto[select]
testing = auto[~select]

In [72]:
sm.ols('mpg~weight_s+year_s+origin_str+cylinders_str', data=training).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.843
Model:                            OLS   Adj. R-squared:                  0.839
Method:                 Least Squares   F-statistic:                     206.8
Date:                Sat, 22 Apr 2023   Prob (F-statistic):          3.26e-119
Time:                        19:55:31   Log-Likelihood:                -814.62
No. Observations:                 318   AIC:                             1647.
Df Residuals:                     309   BIC:                             1681.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             15.2607      1.929      7.911      0.000      11.465      19.056
origin_str[T.2]        1.0946      0.600      1.826      0.069      -0.085       2.274
origin_str[T.3]        1.7111      0.565      3.030      0.003       0.600       2.822
cylinders_str[T.4]     8.1986      1.887      4.345      0.000       4.485      11.912
cylinders_str[T.5]     9.4850      2.702      3.510      0.001       4.167      14.803
cylinders_str[T.6]     5.7990      1.969      2.946      0.003       1.925       9.673
cylinders_str[T.8]     8.0904      2.105      3.844      0.000       3.949      12.231
weight_s              -5.0246      0.429    -11.714      0.000      -5.869      -4.181
year_s                 2.8402      0.197     14.428      0.000       2.453       3.228
==============================================================================
Omnibus:                       29.639   Durbin-Watson:                   1.389
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               51.641
Skew:                           0.559   Prob(JB):                     6.11e-12
Kurtosis:                       4.628   Cond. No.                         32.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [73]:
y_pred = (-5.5893*auto.weight_s)+(2.6460*auto.year_s)+23.4110
np.sum(np.square(y_pred-testing.mpg))

877.904258704606

In [74]:
#Trying to perform logistic regression between origin and other variables. Needs revisiting in order to fix the fact that potential feature
#variables are being missed out
from sklearn.metrics import confusion_matrix
from statsmodels.formula.api import logit

In [75]:
#origin area codes:
#1: usa
#2: germany
#3: japan

In [76]:
ohe = pd.get_dummies(auto, columns=['origin'])
ohe['mpg_s'] = scale(ohe.mpg, with_mean=True, with_std=True)

In [77]:
ohe

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,name,origin_str,cylinders_str,displacement_s,weight_s,acceleration_s,year_s,origin_1,origin_2,origin_3,mpg_s
0,18.0,8,307.0,130.0,3504,12.0,70,chevrolet chevelle malibu,1,8,1.088435,0.630274,-1.294604,-1.626699,1,0,0,-0.705720
1,15.0,8,350.0,165.0,3693,11.5,70,buick skylark 320,1,8,1.500913,0.853457,-1.476652,-1.626699,1,0,0,-1.089551
2,18.0,8,318.0,150.0,3436,11.0,70,plymouth satellite,1,8,1.193953,0.549975,-1.658700,-1.626699,1,0,0,-0.705720
3,16.0,8,304.0,150.0,3433,12.0,70,amc rebel sst,1,8,1.059658,0.546432,-1.294604,-1.626699,1,0,0,-0.961608
4,17.0,8,302.0,140.0,3449,10.5,70,ford torino,1,8,1.040473,0.565326,-1.840747,-1.626699,1,0,0,-0.833664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,27.0,4,140.0,86.0,2790,15.6,82,ford mustang gl,1,4,-0.513513,-0.212865,0.016141,1.629433,1,0,0,0.445772
393,44.0,4,97.0,52.0,2130,24.6,82,vw pickup,2,4,-0.925991,-0.992237,3.293004,1.629433,0,1,0,2.620814
394,32.0,4,135.0,84.0,2295,11.6,82,dodge rampage,1,4,-0.561476,-0.797394,-1.440242,1.629433,1,0,0,1.085491
395,28.0,4,120.0,79.0,2625,18.6,82,ford ranger,1,4,-0.705363,-0.407708,1.108429,1.629433,1,0,0,0.573716


In [78]:
fit1 = logit('origin_1~mpg_s+weight_s+year_s', data=ohe).fit()
fit1.summary()

Optimization terminated successfully.
         Current function value: 0.417868
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               origin_1   No. Observations:                  397
Model:                          Logit   Df Residuals:                      393
Method:                           MLE   Df Model:                            3
Date:                Sat, 22 Apr 2023   Pseudo R-squ.:                  0.3685
Time:                        19:55:32   Log-Likelihood:                -165.89
converged:                       True   LL-Null:                       -262.70
Covariance Type:            nonrobust   LLR p-value:                 1.008e-41
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.0328      0.174      5.944      0.000       0.692       1.373
mpg_s         -0.8064      0.329     -2.451      0.014      -1.451      -0.162
weight_s       1.5164      0.349      4.345      0.000       0.832       2.200
year_s         0.3631      0.190      1.911      0.056      -0.009       0.736
==============================================================================
"""

In [79]:
y_predOrigin1 = fit1.predict()
y_predOrigin1

array([0.87728335, 0.93181542, 0.86356087, 0.88555949, 0.87778715,
       0.97758327, 0.98019331, 0.97867922, 0.98251665, 0.94765097,
       0.91545429, 0.92875198, 0.9391568 , 0.83632708, 0.33645138,
       0.58727341, 0.65920569, 0.50384331, 0.1943453 , 0.1362145 ,
       0.43902494, 0.36001729, 0.31497722, 0.24367923, 0.53111099,
       0.99168875, 0.9873057 , 0.98609351, 0.99390688, 0.21023875,
       0.23384763, 0.28056724, 0.21967639, 0.59974868, 0.89617607,
       0.86474707, 0.83210102, 0.84273195, 0.97681097, 0.98518569,
       0.97447216, 0.97175719, 0.99494752, 0.9918809 , 0.99597385,
       0.74931006, 0.42316321, 0.82703787, 0.80406326, 0.32090315,
       0.19166973, 0.15016711, 0.148122  , 0.08506319, 0.04416424,
       0.13545923, 0.17745585, 0.34289749, 0.26390558, 0.35658681,
       0.4987703 , 0.39317212, 0.98302203, 0.98454588, 0.97349244,
       0.97577605, 0.92877516, 0.99266634, 0.98864851, 0.98887963,
       0.98692298, 0.48964738, 0.95962474, 0.97687751, 0.98360

In [80]:
confusion_matrix(ohe.origin_1, np.round(y_predOrigin1))
#accuracy = 80.1%

array([[110,  39],
       [ 40, 208]], dtype=int64)

In [81]:
fit2 = logit('origin_2~mpg_s+weight_s+year_s', data=ohe).fit()
fit2.summary()
y_predOrigin2 = fit2.predict()
y_predOrigin2

Optimization terminated successfully.
         Current function value: 0.403151
         Iterations 7


array([0.13636336, 0.09888746, 0.14295728, 0.12682104, 0.13332704,
       0.06107174, 0.05659342, 0.05843085, 0.0536103 , 0.08815507,
       0.10863925, 0.09865425, 0.09410307, 0.1430579 , 0.37528961,
       0.26453375, 0.22155983, 0.29020367, 0.4742748 , 0.51616936,
       0.33596728, 0.36437954, 0.39136216, 0.4360132 , 0.28016726,
       0.03536727, 0.04257221, 0.04532683, 0.03015398, 0.43923275,
       0.42995988, 0.3859848 , 0.42132954, 0.22891291, 0.11150186,
       0.12827678, 0.14758803, 0.14030411, 0.05531274, 0.04549123,
       0.0576788 , 0.06027663, 0.02709762, 0.03416738, 0.02509328,
       0.17513957, 0.30559897, 0.14963053, 0.15544748, 0.35466232,
       0.45804768, 0.50304733, 0.50486333, 0.58068045, 0.67620811,
       0.498661  , 0.45674727, 0.3282113 , 0.37210994, 0.31705569,
       0.24919619, 0.29202312, 0.0430239 , 0.04224135, 0.05473388,
       0.05143838, 0.08831185, 0.02839772, 0.03605266, 0.03491337,
       0.03836524, 0.24780253, 0.06581401, 0.0492678 , 0.04236

In [82]:
confusion_matrix(ohe.origin_2, np.round(y_predOrigin2))

array([[324,   3],
       [ 66,   4]], dtype=int64)

In [83]:
fit3 = logit('origin_3~mpg_s+weight_s+year_s', data=ohe).fit()
fit2.summary()
y_predOrigin3 = fit3.predict()
y_predOrigin3

Optimization terminated successfully.
         Current function value: 0.357527
         Iterations 8


array([1.18687335e-02, 7.43624399e-03, 1.41546020e-02, 1.44912777e-02,
       1.37946077e-02, 1.36609400e-03, 1.33091101e-03, 1.48576038e-03,
       1.10491558e-03, 4.93757577e-03, 1.04280394e-02, 9.32672139e-03,
       6.22848825e-03, 3.58112876e-02, 1.82545367e-01, 6.33879617e-02,
       7.54237662e-02, 1.15123336e-01, 2.91503181e-01, 4.73535765e-01,
       9.15831072e-02, 1.60919477e-01, 1.80190498e-01, 2.39936517e-01,
       9.97933451e-02, 6.93175179e-04, 1.29708086e-03, 1.26669765e-03,
       5.14097988e-04, 3.10833583e-01, 2.39420136e-01, 2.61647221e-01,
       3.63590730e-01, 1.13554376e-01, 1.56195835e-02, 2.05766192e-02,
       2.17121895e-02, 2.26828356e-02, 2.13281920e-03, 1.09341370e-03,
       2.46316411e-03, 2.86694725e-03, 3.06531847e-04, 5.26146688e-04,
       1.87160909e-04, 5.17742050e-02, 1.84549382e-01, 2.28554434e-02,
       3.31759591e-02, 2.68852028e-01, 3.13061863e-01, 3.37773595e-01,
       3.43076638e-01, 5.27142413e-01, 6.21660662e-01, 4.95137611e-01,
      

In [84]:
confusion_matrix(ohe.origin_3, np.round(y_predOrigin3))

array([[297,  21],
       [ 52,  27]], dtype=int64)